# Process Control and Safety: A Unified Approach

**Objective:** This lesson demonstrates the critical relationship between process control and process safety. We will design a feedback control loop for a common process unit (a surge tank) and then analyze its performance and safety margins during a process upset.

**Learning Goals:**
1.  Understand the components of a feedback control loop: Process Variable (PV), Setpoint (SP), and Manipulated Variable (MV).
2.  Implement a dynamic model of a simple process (tank level).
3.  Understand and code a Proportional-Integral-Derivative (PID) controller, the workhorse of the process industry.
4.  Simulate the control system's response to a setpoint change and a disturbance.
5.  Integrate a layer of safety by defining and visualizing process alarms.

## The System: Surge Tank Level Control

Our goal is to maintain the liquid level in a surge tank at a desired setpoint. The tank absorbs fluctuations in an incoming stream ($F_{in}$) to provide a steady outgoing stream ($F_{out}$) to a downstream process unit.

*   **Process Variable (PV):** The tank level, $h$.
*   **Setpoint (SP):** The desired tank level, $h_{sp}$.
*   **Manipulated Variable (MV):** The outlet flow, $F_{out}$, which we will control by adjusting a valve.

![Control Loop Diagram](https://www.chemengonline.com/wp-content/uploads/2017/03/46fig5.jpg)

## Part 1: The Process Model - Unsteady-State Mass Balance

To simulate the process, we need a dynamic model. This comes from a simple mass balance on the liquid in the tank:
$$ \text{Accumulation} = \text{In} - \text{Out} $$
In terms of volumetric flow and tank geometry:
$$ \frac{dV}{dt} = F_{in} - F_{out} $$
Since the volume of liquid in a cylindrical tank is $V = A \cdot h$ (where $A$ is the cross-sectional area and $h$ is the level), and assuming a constant area $A$:
$$ A \frac{dh}{dt} = F_{in} - F_{out} $$
Rearranging gives us our final ODE for the process variable:
$$ \frac{dh}{dt} = \frac{1}{A} (F_{in} - F_{out}) $$

## Part 2: The Controller Model - The PID Algorithm

The PID controller calculates how to adjust the manipulated variable ($F_{out}$) based on the error between the setpoint and the process variable. The error is defined as $e(t) = SP - PV$.

The controller output has three components:
*   **Proportional (P):** Reacts to the *current* error. A larger error results in a larger control action. It provides the immediate response.
$$ \text{P-term} = K_c \cdot e(t) $$
*   **Integral (I):** Reacts to the *accumulated past* error. This term is crucial for eliminating any final steady-state error (offset) and forcing the PV to exactly match the SP.
$$ \text{I-term} = \frac{K_c}{\tau_I} \int_{0}^{t} e(t) \,dt $$
*   **Derivative (D):** Reacts to the *rate of change* of the error. It's an anticipatory action that helps to dampen oscillations and prevent overshoot.
$$ \text{D-term} = K_c \tau_D \frac{de(t)}{dt} $$

The controller output is the sum of these actions. The three tuning parameters ($K_c, \tau_I, \tau_D$) determine the aggressiveness and stability of the control action.

## Part 3: Python Implementation and Simulation

We will now combine the process model and the controller model into a single simulation function. Our function will need to keep track of two state variables: the level ($h$) and the accumulated integral error.

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# --- Define Process and Control Parameters ---
# Process Parameters
A = 1.0           # Tank cross-sectional area (m^2)
F_in_initial = 1.0 # Initial inlet flow rate (m^3/min)
h_initial = 2.0     # Initial level in the tank (m)

# Controller Parameters
h_sp = 3.0        # Level Setpoint (m)
Kc = 2.0          # Controller Gain
tau_I = 1.5       # Integral Time (min)
tau_D = 0.2       # Derivative Time (min)

# Disturbance Parameters (for later)
disturbance_time = 20.0 # Time when disturbance occurs (min)
F_in_disturbance = 0.5 # Change in inlet flow (m^3/min)

print("Parameters are defined.")

In [ ]:
# Global lists to store controller output for plotting
F_out_history = []

def tank_control_model(t, y):
    """
    Defines the system of ODEs for the tank level control system.
    
    Args:
        t (float): Current time.
        y (list or array): State vector [level, integral_error].

    Returns:
        list: A list of the derivatives [dh/dt, d(integral_error)/dt].
    """
    # Unpack state variables
    h, integral_error = y
    
    # --- DISTURBANCE --- 
    # Introduce a step change in F_in at the specified time
    if t < disturbance_time:
        F_in = F_in_initial
    else:
        F_in = F_in_initial + F_in_disturbance
    
    # --- CONTROLLER LOGIC --- 
    # 1. Calculate Error
    error = h_sp - h
    
    # 2. Calculate Controller Output (F_out)
    # We need dh/dt to calculate the D-term. We can get it from the model itself.
    # First, calculate a temporary F_out based on P and I terms to avoid circular reference
    F_out_guess = F_in_initial + Kc * error + (Kc / tau_I) * integral_error
    dhdt_approx = (F_in - F_out_guess) / A
    # The error derivative is -dh/dt
    dedt = -dhdt_approx 
    
    # Full PID equation
    # The F_in_initial term is the "bias" or steady-state output
    F_out = F_in_initial + Kc * error + (Kc / tau_I) * integral_error + Kc * tau_D * dedt
    
    # 3. Apply Constraints (valve can't be more than fully open or fully closed)
    # Let's assume max flow is 3.0 m^3/min
    F_out = max(0.0, min(F_out, 3.0))
    F_out_history.append(F_out) # Store for plotting
    
    # --- PROCESS MODEL --- 
    # 4. Calculate the state derivatives
    dh_dt = (F_in - F_out) / A
    d_integral_error_dt = error
    
    return [dh_dt, d_integral_error_dt]

print("Control system model function is defined.")

In [ ]:
# --- Running the Simulation ---

# Reset history for clean runs
F_out_history = []

# Define the time span for the simulation
t_final = 40 # minutes
t_span = (0, t_final)

# Define the initial condition vector [h_initial, integral_error_initial]
y_initial = [h_initial, 0.0]

# Call the ODE solver
solution = solve_ivp(tank_control_model, t_span, y_initial, dense_output=True, method='LSODA')

# Generate points for plotting
t_plot = np.linspace(t_span[0], t_span[1], 500)
y_plot = solution.sol(t_plot)

# Extract individual profiles
h_t = y_plot[0]
# We need to regenerate F_out history for the dense plot points
F_out_plot = []
for i in range(len(t_plot)):
    # Simplified calculation for plotting purposes
    error = h_sp - h_t[i]
    integral_error = y_plot[1][i]
    # Not calculating D-term here for simplicity, but it was used in the solver
    f_out = F_in_initial + Kc * error + (Kc / tau_I) * integral_error
    F_out_plot.append(max(0.0, min(f_out, 3.0)))

print("Simulation complete.")

## Part 4: Analyzing Control Performance and Safety

Let's visualize the results. The key question for a control engineer is: "How well did my controller handle the disturbance?" The key question for a safety engineer is: "Did the process stay within safe operating limits during the disturbance?"

In [ ]:
# --- Define Safety Limits ---
LAH = 3.8  # Level Alarm High (m)
LAHH = 4.0 # Level Alarm High-High (m) - This would typically trigger a shutdown (interlock)
LAL = 2.2  # Level Alarm Low (m)

# --- Plotting the Results ---
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(14, 8))

# Plot Level (PV) and Setpoint (SP)
ax1.set_xlabel('Time (minutes)', fontsize=14)
ax1.set_ylabel('Tank Level (m)', fontsize=14, color='royalblue')
ax1.plot(t_plot, h_t, label='Tank Level (PV)', color='royalblue', linewidth=3)
ax1.axhline(y=h_sp, color='green', linestyle='-', label='Setpoint (SP)')
ax1.tick_params(axis='y', labelcolor='royalblue')

# Plot Safety Alarms
ax1.axhline(y=LAH, color='darkorange', linestyle='--', label=f'High Alarm (LAH)')
ax1.axhline(y=LAHH, color='red', linestyle='--', label=f'High-High Alarm (LAHH)')
ax1.axhline(y=LAL, color='goldenrod', linestyle='--', label=f'Low Alarm (LAL)')

# Mark the disturbance
ax1.axvline(x=disturbance_time, color='black', linestyle=':', label='Disturbance (F_in increases)')
ax1.legend(loc='upper left')
ax1.set_ylim(1.5, 4.5)

# Plot Controller Output on a secondary axis
ax2 = ax1.twinx()
ax2.set_ylabel('Outlet Flow ($m^3/min$)', fontsize=14, color='firebrick')
ax2.plot(t_plot, F_out_plot, label='Outlet Flow (MV)', color='firebrick', alpha=0.7)
ax2.tick_params(axis='y', labelcolor='firebrick')
ax2.legend(loc='upper right')

fig.suptitle('Tank Level Control & Safety Analysis', fontsize=18, weight='bold')
plt.show()

### Interpreting the Plot

1.  **Initial Response:** At the start, the controller sees the level is below the setpoint, so it reduces the outlet flow to allow the level to rise. It successfully brings the level to the setpoint of 3.0 m.
2.  **Disturbance Rejection:** At t=20 minutes, the inlet flow suddenly increases. This is a process upset. The level begins to rise, pushing the PV away from the SP.
3.  **Controller Action:** The PID controller immediately responds to this deviation. It opens the outlet valve (increases $F_{out}$) to counteract the extra inlet flow and drive the level back down towards the setpoint.
4.  **Safety Analysis:** The most important observation from a safety perspective is that **the peak level during the upset remained below the High Alarm (LAH) limit.** This means our primary control loop successfully handled the disturbance without requiring operator intervention or an automated safety shutdown. The system is robust and safe for this particular upset.

## Part 5: You are the Control & Safety Engineer!

The most common task for a control engineer is "tuning" the controller. The goal is to get a fast response with minimal overshoot and oscillation. Modify the **Controller Parameters** in **Part 3** and re-run the simulation to see the effects.

#### Challenge 1: Aggressive Control
Increase the controller gain `Kc` to `5.0`. What happens to the response? Is it faster? Does it oscillate? Does it get dangerously close to the High Alarm during the disturbance? A highly aggressive controller can sometimes be less safe.

#### Challenge 2: Sluggish Control
Reset `Kc` to 2.0. Now, make the integral action much slower by increasing `tau_I` to `10.0`. How well does the controller handle the disturbance now? Does it take a long time to return to the setpoint? What are the risks associated with a sluggish controller during a major upset?

#### Challenge 3: Alarm Rationalization
Set the controller back to its original tuning. Now, imagine the process is very sensitive and the LAH was set at `3.2` m. Re-run the simulation. You'll see the controller generates a "nuisance alarm" even though it's doing its job correctly. This is a real problem in plants, as it can lead to operators ignoring important alarms. This exercise is part of **Alarm Management**, a key safety discipline.